[Jobs](http://127.0.0.1:4040/jobs/)

[Spark master](http://127.0.0.1:8080/)

In [29]:
import pyspark
import pandas as pd
from pyspark.sql import SQLContext
sc = pyspark.SparkContext('spark://spark:7077')
sqlContext = SQLContext(sc)

In [30]:
urls = [
    'https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-04.csv',
    'https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-03.csv',
    'https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-02.csv',
    'https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-01.csv']

pdf = pd.DataFrame({'url': urls})
df = sqlContext.createDataFrame(pdf)

In [31]:
import pandas as pd
import time
from pyspark.sql.functions import pandas_udf, PandasUDFType

df = sqlContext.createDataFrame(pdf)

headers = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-04.csv', nrows=1)
schema = sqlContext.createDataFrame(headers).schema

def download_files(data):
    frames = [pd.read_csv(x) for x in data['url']] 
    return  pd.concat(frames)

df.groupby('url') \
    .applyInPandas(download_files, schema=schema) \
    .limit(100000) \
    .write.partitionBy("Dispatching_base_num") \
    .mode("overwrite") \
    .csv("data/example.csv", header=True)


In [32]:
sqlContext.read.format('csv').options(header='true').load('data/example.csv').show()

+-------------------+----------+--------------------+
|        Pickup_date|locationID|Dispatching_base_num|
+-------------------+----------+--------------------+
|2015-01-01 00:03:00|      null|              B00225|
|2015-01-01 00:03:00|      null|              B00225|
|2015-01-01 00:04:00|      null|              B00225|
|2015-01-01 00:05:00|      null|              B00225|
|2015-01-01 00:06:00|      null|              B00225|
|2015-01-01 00:06:00|      null|              B00225|
|2015-01-01 00:06:00|      null|              B00225|
|2015-01-01 00:09:00|      null|              B00225|
|2015-01-01 00:10:00|      null|              B00225|
|2015-01-01 00:11:00|      null|              B00225|
|2015-01-01 00:11:00|      null|              B00225|
|2015-01-01 00:12:00|      null|              B00225|
|2015-01-01 00:15:00|      null|              B00225|
|2015-01-01 00:17:00|      null|              B00225|
|2015-01-01 00:19:00|      null|              B00225|
|2015-01-01 00:20:00|      n

In [33]:
! ls ./data/example.csv

'Dispatching_base_num=B00013'  'Dispatching_base_num=B00225'
'Dispatching_base_num=B00014'  'Dispatching_base_num=B00227'
'Dispatching_base_num=B00053'  'Dispatching_base_num=B00248'
'Dispatching_base_num=B00095'   _SUCCESS


In [34]:
sc.stop()